# Figure 2 Final

Notebook to make figure 2 for ms <br>
by Cascade Tuholske 2020.02.23

In [1]:
#### Depdencies 
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
from scipy.optimize import fmin
from scipy.stats import beta
from scipy.special import gamma as gammaf
import scipy


In [2]:
#### Functions
def OLS(df, geog, col, alpha, scale):
    
    """Finds linear coef for increase in stat by a given geography from 1983 - 2016, as well
    as the pct change in population of the cities within the given geography
    
    NOTE 2020.03.01 - This will throw a run time warning if all values of a col are zero (e.g. can regress
    a bunch of zeros) ... See note in run_OLD. CPT 
    
    
    Args:
        df = HI stats dataframe
        geog = subset geography to calc people days regression
        col = col to regress on 
        alpha = ci alpha for coef
        scale = to scale data back to original values (usually 10**9) ... see 7_Fig_Data.py
    """

    # Get results
    labels = []
    #delt_list = [] #CPT 2020.02.26
    #r2_list = [] #CPT 2020.02.26
    coef_list = []
    leftci_list = []
    rightci_list = []
    p_list = []
    df_out = pd.DataFrame()

    for label, df_geog in df.groupby(geog):

        # Get Data
        X_year = np.array(df_geog.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
        Y_stats = np.array(df_geog.groupby('year')[col].sum()).reshape((-1, 1))* scale

        # Add Intercept
        X_year_2 = sm.add_constant(X_year)

        # Regress
        model = sm.OLS(Y_stats, X_year_2).fit() 
        
        # Get slope
        # first param in intercept coef, second is slope of line but if slope = 0, then intecept
        if len(model.params) == 2:
            coef = model.params[1]
            
        else:
            coef = model.params[0]
        
        # R2 and P
        #r2 = model.rsquared_adj #CPT 2020.02.26
        p = model.pvalues[0]

        # Pop change #CPT 2020.02.26
#         delt = df_geog.drop_duplicates('ID_HDC_G0').copy()
#         delt['delt_pop'] = delt['P2016'] - delt['P1983']
#         delt = delt['delt_pop'].sum()

        # GET Left and Right CI
        left_ci = model.conf_int(alpha=alpha)[1][0]
        right_ci = model.conf_int(alpha=alpha)[1][1]
        
        # Make lists
        labels.append(label)
        #r2_list.append(r2) #CPT 2020.02.26
        coef_list.append(coef)
        p_list.append(p)
        leftci_list.append(left_ci)
        rightci_list.append(right_ci)
        #delt_list.append(delt) #CPT 2020.02.26

    # Make data frame
    df_out[geog] = labels
    #df_out['p_delt'] = delt_list #CPT 2020.02.26
    #df_out['r2'] = r2_list #CPT 2020.02.26
    df_out['coef'] = coef_list
    df_out['p_value'] = [round(elem, 4) for elem in p_list]
    df_out['ci_left'] = leftci_list
    df_out['ci_right'] = rightci_list 

    return df_out


In [3]:
def run_OLS(stats, geog, alpha, scale):
    """ Function calculate OLS coef of people days due to pop and heat and the 
    attribution index for distribution plots.
    
        
    NOTE 2020.03.01 - This will throw a run time warning if all values of a col are zero (e.g. can regress
    a bunch of zeros, now can we). This will happen if people_days, people_days_pop, people_days_heat or 
    total_days is zero for all years for a given city. This is still OK for our analysis. What is happening is
    that for some cities, the people-days due to heat is zero, meaning pday increases in only due to population. 
    
    This is because with the GHS-UCDB some city's population in 1983 is zero, which forces the pdays due to heat
    to be zero.
    
    -- CPT  
    
    Args:
        stats = df to feed in
        geog = geography level to conduct analysis (city-level is 'ID-HDC-G0')
        alpha = alpha for CI coef 
        scale = to scale data back to original values (usually 10**9) ... see 7_Fig_Data.py
    
    """
    # Get coef for people days
    out = OLS(stats, geog, 'people_days', alpha = alpha, scale = scale)
    out.rename(columns={"coef": "coef_pdays"}, inplace = True)
    out.rename(columns={"p_value": "p_value_pdays"}, inplace = True)
    out.rename(columns={"ci_left": "ci_left_pdays"}, inplace = True)
    out.rename(columns={"ci_right": "ci_right_pdays"}, inplace = True)
    
    # Get people days due to heat coef
    heat = OLS(stats, geog, 'people_days_heat', alpha = alpha, scale = scale) # get stats 
    heat.rename(columns={"coef": "coef_heat"}, inplace = True)
    heat.rename(columns={"p_value": "p_value_heat"}, inplace = True)
    heat.rename(columns={"ci_left": "ci_left_heat"}, inplace = True)
    heat.rename(columns={"ci_right": "ci_right_heat"}, inplace = True)
    out = out.merge(heat, on = geog, how = 'left') # merge
    
    # Get people days due to pop
    pop = OLS(stats, geog, 'people_days_pop', alpha = alpha, scale = scale) # get stats 
    pop.rename(columns={"coef": "coef_pop"}, inplace = True)
    pop.rename(columns={"p_value": "p_value_pop"}, inplace = True)
    pop.rename(columns={"ci_left": "ci_left_pop"}, inplace = True)
    pop.rename(columns={"ci_right": "ci_right_pop"}, inplace = True)
    out = out.merge(pop, on = geog, how = 'left') # merge
    
    # Get total days
    totDays = OLS(stats, geog, 'total_days', alpha = alpha, scale = scale) # get stats 
    totDays.rename(columns={"coef": "coef_totDays"}, inplace = True)
    totDays.rename(columns={"p_value": "p_value_totDays"}, inplace = True)
    totDays.rename(columns={"ci_left": "ci_left_totDays"}, inplace = True)
    totDays.rename(columns={"ci_right": "ci_right_totDays"}, inplace = True)
    out = out.merge(totDays, on = geog, how = 'left') # merge
    
    # attrib coef --- creates range -1 to 1 index of heat vs. population as a driver of total pdays increase
    out['coef_attrib'] = (out['coef_pop'] - out['coef_heat']) / (out['coef_pop'] + out['coef_heat']) # normalize dif
    
    # I am not sure if this works correcetly ... CPT 2020.02.27
    out['coef_attrib_left'] = (out['ci_left_pop'] - out['ci_left_heat']) / (out['ci_left_pop'] + out['ci_left_heat']) # normalize dif
    out['coef_attrib_right'] = (out['ci_right_pop'] - out['ci_right_heat']) / (out['ci_right_pop'] + out['ci_left_heat']) # normalize dif
    
    # drop all neg or zero pday slopes
    out = out[out['coef_pdays'] > 0]
    out = out[out['coef_heat'] > 0]
    out = out[out['coef_pop'] > 0]
    
    # normalize coef of attribution 
    norm = out['coef_attrib']
    out['coef_attrib_norm'] = (norm-min(norm))/(max(norm)-min(norm))
    norm = out['coef_attrib_left']
    out['coef_attrib_norm_left'] = (norm-min(norm))/(max(norm)-min(norm))
    norm = out['coef_attrib_right']
    out['coef_attrib_norm_right'] = (norm-min(norm))/(max(norm)-min(norm))
    
    return out

# LOAD DATA

In [4]:
#### Load Data
# file path
DATA_IN = "/home/cascade/projects/UrbanHeat/data/"  # Note: Need ?dl=1 to make sure this file gets read correctly
FIG_OUT = "/home/cascade/projects/UrbanHeat/figures/"

# Raw Heat
FN_IN = 'processed/All_data_HI406_figdata.csv'
HI_STATS = pd.read_csv(DATA_IN+FN_IN)

# scale the date in the plot 
scale = 10**9 

In [5]:
HI_STATS.head()

,Unnamed: 0,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop
0,0,22,1983,2,52064.452435,52064.452435,73006.671133,0.000104,0.000104,0.0
1,1,26,1983,1,194088.886834,194088.886834,268055.635628,0.000194,0.000194,0.0
2,2,27,1983,1,80540.779940,80540.779940,93335.494324,0.000081,0.000081,0.0
3,3,28,1983,9,59320.971209,59320.971209,91449.606255,0.000534,0.000534,0.0
4,4,29,1983,8,336518.836621,336518.836621,533318.453653,0.002692,0.002692,0.0


In [6]:
# cols we want to add to HI_STATS
cols = ['region', 'intermediate-region', 'sub-region','CTR_MN_NM', 'ID_HDC_G0'] 

# open all the data
meta_fn = 'processed/All_data_HI406_meta.csv' # open all the data
all_data = pd.read_csv(DATA_IN+meta_fn)

# drop ID duplicates
meta = all_data.drop_duplicates('ID_HDC_G0')
meta = meta[cols]

In [7]:
print(len(all_data))
print(len(all_data.drop_duplicates('ID_HDC_G0')))
print(len(HI_STATS))
print(len(meta))

5616932
11558
392972
11558


In [8]:
HI_STATS = HI_STATS.merge(meta, on = 'ID_HDC_G0', how = 'inner')
print('how long is heat stats?',len(HI_STATS))
HI_STATS.head(2)

how long is heat stats? 392972


,Unnamed: 0,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop,region,intermediate-region,sub-region,CTR_MN_NM
0,0,22,1983,2,52064.452435,52064.452435,73006.671133,0.000104,0.000104,0.000000,Americas,Northern America,Northern America,United States
1,10112,22,1984,1,53373.163358,52064.452435,73006.671133,0.000053,0.000052,0.000001,Americas,Northern America,Northern America,United States


# TEST FUNCTIONS

First time run_OLS it throws a warning, but if you run it again it is fine ...

Beta fit for Western Asia throws an error because at some point python reduces a very large neg exp to zero and tries to divide

In [34]:
Y = np.array([0,0,0,0,0,0,0,0,0]).reshape((-1, 1))
X = np.array([0,2,3,4,5,6,7,10, 11]).reshape((-1, 1))* scale

In [35]:
# Add Intercept
X_2 = sm.add_constant(X)

# Regress
model = sm.OLS(Y, X_2).fit() 

In [36]:
model.summary()

/home/cascade/miniconda3/envs/geo/lib/python3.6/site-packages/scipy/stats/stats.py:1450: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  "anyway, n=%i" % int(n))


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sun, 01 Mar 2020   Prob (F-statistic):                nan
Time:                        11:58:27   Log-Likelihood:                    inf
No. Observations:                   9   AIC:                              -inf
Df Residuals:                       7   BIC:                              -inf
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const               0          0        nan        nan           0           0
x1                  0          0        nan        nan           0           0
==============================================================================
Omnibus:                      462.918   Durbin-Watson:                     nan
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                3.375
Skew:                           0.000   Prob(JB):                        0.185
Kurtosis:                       0.000   Cond. No.                     1.18e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.18e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [41]:
#### Global
geog = 'sub-region'
loc = 'Sub-Saharan Africa'##>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> WHY 'Western Asia'
chunk = HI_STATS[HI_STATS[geog] == loc]

In [42]:
OLS(chunk, 'ID_HDC_G0', col = 'total_days', alpha = 0.05, scale = 10**9)

,ID_HDC_G0,coef,p_value,ci_left,ci_right
0,1452,6.485867e+08,0.0000,3.740121e+08,9.231613e+08
1,1453,1.124675e+09,0.0004,5.581256e+08,1.691225e+09
2,1454,1.297479e+09,0.0004,6.468936e+08,1.948064e+09
3,1455,1.331551e+09,0.0003,6.830013e+08,1.980100e+09
4,1456,1.266157e+09,0.0014,5.736955e+08,1.958619e+09
...,...,...,...,...,...
1590,5861,1.750497e+09,0.0000,1.258507e+09,2.242486e+09
1591,5871,9.702063e+08,0.0000,6.914314e+08,1.248981e+09
1592,5873,8.021390e+07,0.0006,3.756217e+07,1.228656e+08
1593,5875,3.529412e+08,0.0005,1.681228e+08,5.377596e+08


In [43]:
OLS(chunk, 'ID_HDC_G0', col = 'people_days', alpha = 0.05, scale = 10**9)

,ID_HDC_G0,coef,p_value,ci_left,ci_right
0,1452,2.523283e+06,0.0000,1.883331e+06,3.163235e+06
1,1453,9.960308e+05,0.0000,8.252756e+05,1.166786e+06
2,1454,9.142742e+05,0.0000,7.134019e+05,1.115146e+06
3,1455,1.800270e+06,0.0000,1.476834e+06,2.123706e+06
4,1456,1.628861e+05,0.0000,1.204896e+05,2.052826e+05
...,...,...,...,...,...
1590,5861,1.573397e+05,0.0000,1.251476e+05,1.895317e+05
1591,5871,7.604855e+05,0.0000,6.987148e+05,8.222563e+05
1592,5873,4.287144e+03,0.0001,2.289459e+03,6.284830e+03
1593,5875,3.662902e+04,0.0000,2.138431e+04,5.187373e+04


In [44]:
OLS(chunk, 'ID_HDC_G0', col = 'people_days_heat', alpha = 0.05, scale = 10**9)

,ID_HDC_G0,coef,p_value,ci_left,ci_right
0,1452,924082.590964,0.0000,532878.731972,1.315286e+06
1,1453,208324.773916,0.0004,103382.185171,3.132674e+05
2,1454,302686.259582,0.0004,150912.515351,4.544600e+05
3,1455,316313.037011,0.0003,162248.578007,4.703775e+05
4,1456,57139.702217,0.0014,25889.981148,8.838942e+04
...,...,...,...,...,...
1590,5861,61861.143647,0.0000,44474.623485,7.924766e+04
1591,5871,106410.295986,0.0000,75834.819148,1.369858e+05
1592,5873,2026.818857,0.0006,949.108831,3.104529e+03
1593,5875,18853.583666,0.0005,8980.862950,2.872630e+04


In [45]:
np.unique(OLS(chunk, 'ID_HDC_G0', col = 'people_days_heat', alpha = 0.05, scale = 10**9)['coef'] == 0)

array([False,  True])

In [46]:
test = OLS(chunk, 'ID_HDC_G0', col = 'people_days_heat', alpha = 0.05, scale = 10**9)
test[test['coef'] == 0]

,ID_HDC_G0,coef,p_value,ci_left,ci_right
216,2031,0.0,NaN,0.0,0.0
1119,3832,0.0,NaN,0.0,0.0
1223,4227,0.0,NaN,0.0,0.0
1335,4532,0.0,NaN,0.0,0.0
1417,4738,0.0,NaN,0.0,0.0
1430,4795,0.0,NaN,0.0,0.0
1446,4862,0.0,NaN,0.0,0.0
1447,4867,0.0,NaN,0.0,0.0
1502,5333,0.0,NaN,0.0,0.0
1516,5424,0.0,NaN,0.0,0.0


In [48]:
chunk[chunk['ID_HDC_G0'] == 4795]

,Unnamed: 0,ID_HDC_G0,year,total_days,P,P1983,P2016,people_days,people_days_heat,people_days_pop,region,intermediate-region,sub-region,CTR_MN_NM
107100,3150,4795,1983,13,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107101,13103,4795,1984,10,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107102,22680,4795,1985,3,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107103,32171,4795,1986,7,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107104,41916,4795,1987,23,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107105,51871,4795,1988,9,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107106,61731,4795,1989,2,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107107,71449,4795,1990,14,0.000000,0.0,56573.515625,0.000000,0.0,0.000000,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107108,81306,4795,1991,6,3600.720313,0.0,56573.515625,0.000022,0.0,0.000022,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique
107109,91007,4795,1992,17,7201.440625,0.0,56573.515625,0.000122,0.0,0.000122,Africa,Eastern Africa,Sub-Saharan Africa,Mozambique


In [ ]:
OLS(chunk, 'ID_HDC_G0', col = 'people_days_pop', alpha = 0.05)

In [ ]:
#### Global
geog = 'sub-region'
loc = 'Western Asia'##>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> WHY 'Western Asia'
chunk = HI_STATS[HI_STATS[geog] == loc]


plotdata = run_OLS(chunk, 'ID_HDC_G0', alpha = 0.05, scale = 10**9)

In [ ]:
plotdata

In [ ]:
OLS(chunk, 'ID_HDC_G0', col = 'total_days', alpha = 0.05)

In [ ]:
# drop all neg or zero pday slopes
plotdata = plotdata[plotdata['coef_pdays'] > 0]
plotdata = plotdata[plotdata['coef_heat'] > 0]
plotdata = plotdata[plotdata['coef_pop'] > 0]
    

In [ ]:
# Try beta function`

betadata = plotdata['coef_attrib_norm']
WA_Beta = beta_fit(betadata)

In [ ]:
# try pieces of beta function
alpha3,beta3,xx,yy=beta.fit(betadata, loc = 0, scale = 1)
xx=np.linspace(0,max(betadata),len(betadata)) # line space of X
xx

In [ ]:
fitted=lambda x,a,b:gammaf(a+b)/gammaf(a)/gammaf(b)*x**(a-1)*(1-x)**(b-1) #pdf of beta


In [ ]:
a = alpha3
b = beta3
print('alpha is ', a)
print('beta is ', b)

In [ ]:
yy = fitted(xx,a,b)

In [ ]:
plt.hist(betadata, bins = 50, alpha = 0.5, density = True)
plt.plot(xx, yy, linewidth = 3)
plt.title('Western Asia May not be a Beta fit...?')

In [ ]:
#----------------Fit using moments----------------
data = betadata
mean=np.mean(data)
var=np.var(data,ddof=1)
alpha1=mean**2*(1-mean)/var-mean
beta1=alpha1*(1-mean)/mean

In [ ]:
beta1

In [ ]:
xx=np.linspace(0,max(betadata),len(betadata)) # line space of X
yy = fitted(xx,alpha1,beta1)

In [ ]:
plt.hist(betadata, bins = 50, alpha = 0.5, density = True)
plt.plot(xx, yy, linewidth = 3)
plt.title('Western Asia May not be a Beta fit...?')

In [ ]:
def betaNLL(param,*args):
    '''Negative log likelihood function for beta
    <param>: list for parameters to be fitted.
    <args>: 1-element array containing the sample data.

    Return <nll>: negative log-likelihood to be minimized.
    '''

    a,b=param
    data=args[0]
    pdf=beta.pdf(data,a,b,loc=0,scale=1)
    lg=np.log(pdf)
    #-----Replace -inf with 0s------
    lg=np.where(lg==-np.inf,0,lg)
    nll=-1*np.sum(lg)
    return nll

#------------------Fit using mle------------------
result=fmin(betaNLL,[1,1],args=(data,))
alpha2,beta2=result

In [ ]:
xx=np.linspace(0,max(betadata),len(betadata)) # line space of X
yy = fitted(xx,alpha2,beta2)
plt.hist(betadata, bins = 50, alpha = 0.5, density = True)
plt.plot(xx, yy, linewidth = 3)
plt.title('Western Asia May not be a Beta fit...?')

In [ ]:
beta_mle(betadata)

# Beta Fits

**RUN OLS IS THROWING A WARNING - BEST LOOK -- but then doesn't ...**

In [ ]:
#### Global
plotdata = run_OLS(HI_STATS, 'ID_HDC_G0', alpha = 0.05)

#### Chunk 1
geog1 = 'sub-region'
loc1 = 'Sub-Saharan Africa'
chunk1 = HI_STATS[HI_STATS[geog1] == loc1]
plotdata1 = run_OLS(chunk1, 'ID_HDC_G0', alpha = 0.05)


#### Chunk 2
geog2 = 'sub-region'
loc2 = 'Eastern Asia'
chunk2 = HI_STATS[HI_STATS[geog2] == loc2]
plotdata2 = run_OLS(chunk2, 'ID_HDC_G0', alpha = 0.05)


#### Chunk 3
geog3 = 'sub-region'
loc3 = 'Western Asia'
chunk3 = HI_STATS[HI_STATS[geog3] == loc3]
plotdata3 = run_OLS(chunk3, 'ID_HDC_G0', alpha = 0.05)

#### Chunk 4
geog4 = 'region'
loc4 = 'Europe'
chunk4 = HI_STATS[HI_STATS[geog4] == loc4]
plotdata4 = run_OLS(chunk4, 'ID_HDC_G0', alpha = 0.05)

In [ ]:
#### Size
fig, axs = plt.subplots(1, 4, figsize = (24, 6), sharex=True)
axs = axs.ravel()
ws = 0.25
fig.subplots_adjust(wspace=ws)
fig = plt.gcf()
xlim = [0,1]
ylim = [0,4]

g_X, g_Y = beta_mle(plotdata['coef_attrib_norm'])

kwargs = {'lw': 3, 'alpha' : 0.9, 'linestyle' : '-'}
kwargs_g = {'lw': 3, 'alpha' : 0.5, 'linestyle' : ':'}

#### Chunk 1
data = plotdata1['coef_attrib_norm']
X, Y = beta_mle(data)
axs[0].hist(data, density = True, color = 'red', alpha = 0.1, bins = 30)
axs[0].plot(X, Y, label = loc1, color = 'red', **kwargs)
axs[0].plot(g_X, g_Y, label = loc1, color = 'black', **kwargs_g)
axs[0].set_title(loc1)
axs[0].set_xlim(xlim)
axs[0].set_ylim(ylim)

# #### Chunk 2
data = plotdata2['coef_attrib_norm']
X, Y = beta_mle(data)
axs[1].hist(data, density = True, color = 'blue', alpha = 0.1, bins = 30)
axs[1].plot(X, Y, label = loc2, color = 'blue', **kwargs)
axs[1].plot(g_X, g_Y, label = loc1, color = 'black', **kwargs_g)
axs[1].set_title(loc2)
axs[1].set_xlim(xlim)
axs[1].set_ylim(ylim)

#### Chunk 3
data = plotdata3['coef_attrib_norm']
X, Y = beta_mle(data)
axs[2].hist(data, density = True, color = 'green', alpha = 0.1, bins = 30)
axs[2].plot(X, Y, label = loc3, color = 'green', **kwargs)
axs[2].plot(g_X, g_Y, label = loc1, color = 'black', **kwargs_g)
axs[2].set_title(loc3)
axs[2].set_xlim(xlim)
axs[2].set_ylim(ylim)

#### Chunk 4
data = plotdata4['coef_attrib_norm']
X, Y = beta_mle(data)
axs[3].hist(data, density = True, color = 'orange', alpha = 0.1, bins = 30)
axs[3].plot(X, Y, label = loc3, color = 'orange', **kwargs)
axs[3].plot(g_X, g_Y, label = loc1, color = 'black', **kwargs_g)
axs[3].set_title(loc4)
axs[3].set_xlim(xlim)
axs[3].set_ylim(ylim)

#### Lims and Ticks
plt.xlim([0,1])
plt.ylim([0,4])
ticks = [x / 10 for x in list(range(0,11))]
labels = [100, 80, 60, 40, 20, 0, 20, 40, 60, 80, 100]
plt.xticks(ticks, labels, fontsize = 12);

#### Legends
# labels = [loc1, loc2, loc3]
# plt.legend(labels, loc=2, facecolor= 'white', edgecolor = 'white', fontsize = 15)


# ## Labels 
# plt.xlabel('Coef of Attribtuion (Right is people, left heat)', fontsize = 15, labelpad = 10)
# plt.ylabel('PDF', fontsize = 15)
# fig.title('Dist of PeopleDays Attribution Coef, 1983 - 2016', fontsize = 15)


In [ ]:
#### Size
fig, axs = plt.subplots(1, 4, figsize = (24, 6), sharex=True)
axs = axs.ravel()
ws = 0.25
fig.subplots_adjust(wspace=ws)


#### Chunk 1
X = np.log10(plotdata1['coef_pop']*scale)
Y = np.log10(plotdata1['coef_heat']*scale)
C = np.log10(plotdata1['coef_pdays']) ##########<<<< CASCADE FIX SO WE CAN CITY

# plots
im  = axs[0].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')

# Make one-to-one-line
axs[0].plot([0, lim], [0, lim], ':', lw=3, alpha = 0.7, color = 'black')
axs[0].set_title(loc1)

#fig.colorbar(ax = axs[4], mappable = im)

# Set limits
axs[0].set_xlim([-0.5, lim])
axs[0].set_ylim([-0.5, lim])

#### Chunk 2
X = np.log10(plotdata2['coef_pop']*scale)
Y = np.log10(plotdata2['coef_heat']*scale)
C = np.log10(plotdata2['coef_pdays']) ##########<<<< CASCADE FIX SO WE CAN CITY

# plots
im  = axs[1].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')

# Make one-to-one-line
axs[1].plot([0, lim], [0, lim], ':', lw=3, alpha = 0.7, color = 'black')
axs[1].set_title(loc2)

#fig.colorbar(ax = axs[4], mappable = im)

# Set limits
axs[1].set_xlim([-0.5, lim])
axs[1].set_ylim([-0.5, lim])

#### Chunk 3
X = np.log10(plotdata3['coef_pop']*scale)
Y = np.log10(plotdata3['coef_heat']*scale)
C = np.log10(plotdata3['coef_pdays']) ##########<<<< CASCADE FIX SO WE CAN CITY

# plots
im  = axs[2].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')

# Make one-to-one-line
axs[2].plot([0, lim], [0, lim], ':', lw=3, alpha = 0.7, color = 'black')
axs[2].set_title(loc3)

#fig.colorbar(ax = axs[4], mappable = im)

# Set limits
axs[2].set_xlim([-0.5, lim])
axs[2].set_ylim([-0.5, lim])

#### Chunk 4
X = np.log10(plotdata4['coef_pop']*scale)
Y = np.log10(plotdata4['coef_heat']*scale)
C = np.log10(plotdata4['coef_pdays']) ##########<<<< CASCADE FIX SO WE CAN CITY

# plots
im  = axs[3].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')

# Make one-to-one-line
axs[3].plot([0, lim], [0, lim], ':', lw=3, alpha = 0.7, color = 'black')
axs[3].set_title(loc4)

#fig.colorbar(ax = axs[4], mappable = im)

# Set limits
axs[3].set_xlim([-0.5, lim])
axs[3].set_ylim([-0.5, lim])

# Fig 2 Scatter

In [ ]:
#### Plot and Save Figure 
#### Plot is people-days heat vs pop increase in log scale values
fig, axs = plt.subplots(5, 3, figsize = (15, 15), sharex=True) # Change number of panels as needed
axs = axs.ravel()
counter = 0
lim = 13

for i, region in enumerate(sig_regions):
    
    df_region = stats[stats[geog] == region] # select region
    
    # Get people days dues to heat coef
    heat = pop_stat(df_region, 'ID_HDC_G0', 'people_days_heat') # get stats 
    heat.rename(columns={"coef": "coef_heat"}, inplace = True)
    
    # Get people days total
    pdays_all = pop_stat(df_region, 'ID_HDC_G0', 'people_days_pop') # get stats
    pdays_all.rename(columns={"coef": "coef_pop"}, inplace = True)
    
    # Merge pdays_all and heat
    heat = heat.merge(pdays_all[['ID_HDC_G0', 'coef_pop']], on = 'ID_HDC_G0', how = 'left')
    
    # drop negitive and zero slopes
    heat = heat[heat['coef_heat'] > 0]
    heat = heat[heat['coef_pop'] > 0]
    
    # get data for plot
    X = np.log10(heat['coef_pop']*scale)
    Y = np.log10(heat['coef_heat']*scale)
    C = np.log10(heat['p_delt'])
    
    # drop all slopes below 10^0
    X = X * (X > 0)
    Y = Y * (Y > 0)
    
    # plots
    a = i + counter
    im  = axs[a].scatter(X, Y, alpha = 0.4, marker = 'h', c = C, cmap = 'rainbow')
    axs[a].set_title(region, fontsize = 14)
    
    # Set limits
    axs[a].set_xlim([-0.5, lim])
    axs[a].set_ylim([-0.5, lim])
    
    # Make one-to-one-line
    axs[a].plot([0, lim], [0, lim], ':', lw=3, alpha = 0.7, color = 'black')
    
    fig.colorbar(ax = axs[a], mappable = im)

fig.text(0.5, 0.08, 'Annual increase in people-days [log10] due to population', ha='center', size = 16)
fig.text(0.07, 0.5, 'Annual increase in people-days [log10] due to heat', va='center', rotation='vertical',
        size = 16)

#plt.savefig(FIG_OUT+'FIG2.png') 

# Fig 2 Dist

In [ ]:
def plot_data(stats, geog):
    """ Function calculate coef of people days due to pop and heat and the 
    attribution index
    
    Args:
        stats = df to feed in
        geog = geography level to conduct analysis (city-level is 'ID-HDC-G0')
    
    """
    # Get people days dues to heat coef
    heat = pop_stat(stats, geog, 'people_days_heat') # get stats 
    heat.rename(columns={"coef": "coef_heat"}, inplace = True)
    
    # Get people days total
    pdays_all = pop_stat(stats, geog, 'people_days_pop') # get stats
    pdays_all.rename(columns={"coef": "coef_pop"}, inplace = True)
    
    # Merge pdays_all and heat
    heat = heat.merge(pdays_all[[geog, 'coef_pop']], on = geog, how = 'left')
    
    # drop negitive and zero slopes
    heat = heat[heat['coef_heat'] > 0]
    heat = heat[heat['coef_pop'] > 0]
    
    # attrib coef
    heat['coef_attrib'] = (heat['coef_pop'] - heat['coef_heat']) / (heat['coef_pop'] + heat['coef_heat']) # normalize dif
    
    return heat

In [ ]:
#### Chunk Data
geog = 'sub-region'
loc = 'Northern America'
chunk_stats = stats[stats[geog] == loc]

Y95= chunk_stats[(chunk_stats['year'] >= 1986) & (chunk_stats['year'] <= 1995)]
Y05 = chunk_stats[(chunk_stats['year'] >= 1996) & (chunk_stats['year'] <= 2005)]
Y15 = chunk_stats[(chunk_stats['year'] >= 2006) & (chunk_stats['year'] <= 2015)]

In [ ]:
#### Make distribution

Y95_plotdata = plot_data(Y95, 'ID_HDC_G0')
Y95_plotdata.head()

Y05_plotdata = plot_data(Y05, 'ID_HDC_G0')
Y95_plotdata.head()

Y15_plotdata = plot_data(Y15, 'ID_HDC_G0')
Y15_plotdata.head()

In [ ]:
data = Y15_plotdata['coef_attrib']
bins = 20
#plt.hist(data, color = 'blue', alpha = 0.3, bins = bins)
#sns.kdeplot(np.array(data), bw=0.15, color = 'blue', alpha = 0.4)

data = Y05_plotdata['coef_attrib']
#plt.hist(data, color = 'green', alpha = 0.3, bins = bins)
#sns.kdeplot(np.array(data), bw=0.15, color = 'green', alpha = 0.4)

data = Y95_plotdata['coef_attrib']
plt.hist(data, color = 'red', alpha = 0.3, bins = bins)
#sns.kdeplot(np.array(data), bw=0.15, color = 'red', alpha = 0.4)
plt.xlabel('Coef of Attribtuion (Right is people, left heat)')
plt.ylabel('N of Cites')
plt.title(loc+' Dist of PeopleDays Attribution Coef')
#sns.kdeplot(np.array(data), bw=0.5)

labels = ['1986 - 1995', '1996 - 2005', '2006 - 2015']
plt.legend(labels, loc=1, facecolor= 'white', edgecolor = 'white')


# Desnity Plots


### North America 

In [ ]:
#### Chunk Data
geog = 'region'
loc = 'Europe'
chunk_stats = stats[stats[geog] == loc]

Y95= chunk_stats[(chunk_stats['year'] >= 1986) & (chunk_stats['year'] <= 1995)]
Y05 = chunk_stats[(chunk_stats['year'] >= 1996) & (chunk_stats['year'] <= 2005)]
Y15 = chunk_stats[(chunk_stats['year'] >= 2006) & (chunk_stats['year'] <= 2015)]

#### Make distribution
all_plotdata = plot_data(chunk_stats, 'ID_HDC_G0')
Y95_plotdata = plot_data(Y95, 'ID_HDC_G0')
Y05_plotdata = plot_data(Y05, 'ID_HDC_G0')
Y15_plotdata = plot_data(Y15, 'ID_HDC_G0')

In [ ]:
bw = 'scott'
kws1 = {'bw': bw, 'linewidth': 0, 'shade': True, 'alpha' : 0.05}
kws2 = {'bw': bw, 'linewidth': 2, 'shade': False, 'alpha' : 0.5}

#### Size
fig = plt.gcf()
fig.set_size_inches(8, 8)

data = all_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'black', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'black', kde_kws= kws2, label = '1983 - 2016')

data = Y15_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws2, label = '2006 - 2015')


data = Y05_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws2, label = '1996 - 2005')

data = Y95_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws2, label = '1986 - 1995')


## Labels 
plt.xlabel('Coef of Attribtuion (Right is people, left heat)')
plt.ylabel('Density')
plt.title(loc+' Dist of PeopleDays Attribution Coef')

## Legend
#labels = ['1986 - 1995', '1996 - 2005', '2006 - 2015']
plt.legend(loc=2, facecolor= 'white', edgecolor = 'white')

### SS Africa

In [ ]:
#### Chunk Data
geog = 'sub-region'
loc = 'Sub-Saharan Africa'
chunk_stats = stats[stats[geog] == loc]

Y95= chunk_stats[(chunk_stats['year'] >= 1986) & (chunk_stats['year'] <= 1995)]
Y05 = chunk_stats[(chunk_stats['year'] >= 1996) & (chunk_stats['year'] <= 2005)]
Y15 = chunk_stats[(chunk_stats['year'] >= 2006) & (chunk_stats['year'] <= 2015)]

#### Make distribution
Y95_plotdata = plot_data(Y95, 'ID_HDC_G0')
Y05_plotdata = plot_data(Y05, 'ID_HDC_G0')
Y15_plotdata = plot_data(Y15, 'ID_HDC_G0')

In [ ]:
bw = 'scott'
kws = {'bw': bw, 'linewidth': 1, 'alpha' : 0.5}

#### Size
fig = plt.gcf()
fig.set_size_inches(8, 8)

data = Y15_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws, label = '2006 - 2015')

data = Y05_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws, label = '1996 - 2005')

data = Y95_plotdata['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws, label = '1986 - 1995')

## Labels 
plt.xlabel('Coef of Attribtuion (Right is people, left heat)')
plt.ylabel('Density')
plt.title(loc+' Dist of PeopleDays Attribution Coef')

## Legend
#labels = ['1986 - 1995', '1996 - 2005', '2006 - 2015']
plt.legend(loc=2, facecolor= 'white', edgecolor = 'white')

## Plots of Different Regions

In [ ]:
#### Chunk SSA
geog = 'sub-region'
loc = 'Sub-Saharan Africa'
chunk_SSA = stats[stats[geog] == loc]
plotdata_SSA = plot_data(chunk_SSA, 'ID_HDC_G0')

#### Chunk SSA
geog = 'region'
loc = 'Europe'
chunk_EU = stats[stats[geog] == loc]
plotdata_EU = plot_data(chunk_EU, 'ID_HDC_G0')

#### Chunk North America
geog = 'sub-region'
loc = 'Northern America'
chunk_NA = stats[stats[geog] == loc]
plotdata_NA = plot_data(chunk_NA, 'ID_HDC_G0')

In [ ]:
bw = 'scott'
kws1 = {'bw': bw, 'linewidth': 0, 'shade': True, 'alpha' : 0.05}
kws2 = {'bw': bw, 'linewidth': 2, 'shade': False, 'alpha' : 0.5}

#### Size
fig = plt.gcf()
fig.set_size_inches(8, 8)

data = plotdata_SSA['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws2, label = 'Sub-Saharan Africa')

data = plotdata_EU['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws2, label = 'Europe')

data = plotdata_NA['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws2, label = 'Northern America')


## Labels 
plt.xlabel('Coef of Attribtuion (Right is people, left heat) 1983 - 2016')
plt.ylabel('Density')
plt.title(loc+' Dist of PeopleDays Attribution Coef')

## Legend
#labels = ['1986 - 1995', '1996 - 2005', '2006 - 2015']
plt.legend(loc=2, facecolor= 'white', edgecolor = 'white')

In [ ]:
#### Size
fig = plt.gcf()
fig.set_size_inches(8, 8)

data = plotdata_SSA['coef_attrib']
plt.hist(data, color = 'r', alpha = 0.5)

data = plotdata_EU['coef_attrib']
plt.hist(data, color = 'b', alpha = 0.5)

data = plotdata_NA['coef_attrib']
plt.hist(data, color = 'g', alpha = 0.5)


In [ ]:
#### Chunk SSA
geog = 'CTR_MN_NM'
loc1 = 'Saudi Arabia'
chunk_1 = stats[stats[geog] == loc1]
plotdata_1 = plot_data(chunk_1, 'ID_HDC_G0')

#### Chunk SSA
geog = 'CTR_MN_NM'
loc2 = 'Mexico'
chunk_2 = stats[stats[geog] == loc2]
plotdata_2 = plot_data(chunk_2, 'ID_HDC_G0')

#### Chunk North America
geog = 'CTR_MN_NM'
loc3 = 'China'
chunk_3 = stats[stats[geog] == loc3]
plotdata_3 = plot_data(chunk_3, 'ID_HDC_G0')

In [ ]:
bw = 'scott'
kws1 = {'bw': bw, 'linewidth': 0, 'shade': True, 'alpha' : 0.05}
kws2 = {'bw': bw, 'linewidth': 2, 'shade': False, 'alpha' : 0.5}

#### Size
fig = plt.gcf()
fig.set_size_inches(8, 8)

data = plotdata_1['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'darkblue', kde_kws= kws2, label = loc1)

data = plotdata_2['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'red', kde_kws= kws2, label = loc2)

data = plotdata_3['coef_attrib']
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws1)
sns.distplot(data, hist=False, kde=True, color = 'green', kde_kws= kws2, label = loc3)


## Labels 
plt.xlabel('Coef of Attribtuion (Right is people, left heat) 1983 - 2016')
plt.ylabel('Density')
plt.title(loc+' Dist of PeopleDays Attribution Coef')

## Legend
#labels = ['1986 - 1995', '1996 - 2005', '2006 - 2015']
plt.legend(loc=2, facecolor= 'white', edgecolor = 'white')

# Old OLS TEST

In [ ]:
#### Chunk SSA
geog = 'CTR_MN_NM'
loc = 'Iran'
chunk = HI_STATS[HI_STATS[geog] == loc]


In [ ]:
chunk.head()

In [ ]:
alpha = 0.05
test_ols = OLS_coef(chunk, geog = 'ID_HDC_G0', col = 'total_days', alpha = alpha)
test_ols = test_ols[test_ols['p_value'] < 0.05]
test_ols.head(1)

In [ ]:
# Get Data
df_geog =  HI_STATS[HI_STATS['ID_HDC_G0'] == 5368]
col = 'total_days'

X_year = np.array(df_geog.groupby('year')['ID_HDC_G0'].mean().index).reshape((-1, 1))
Y_stats = np.array(df_geog.groupby('year')[col].sum()).reshape((-1, 1))

# Add Intercept
X_year_2 = sm.add_constant(X_year)

# Regress
model = sm.OLS(Y_stats, X_year_2).fit() 

In [ ]:
print(model.conf_int(alpha=alpha)[1][0])
print(model.conf_int(alpha=alpha)[1][1])
print(round(model.params[1], 7))
print(model.rsquared_adj)

In [ ]:
model.summary()